In [2]:
import pandas as pd 
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
label = pd.read_csv('devset_images_gt.csv') 
# Preview the first 5 lines of the loaded data 
label_array = label.values


In [4]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Đường dẫn tới folder chứa ảnh
folder_path = "C:/Users/minhd/FPTU lab/DPL302m/Kaggle/2024-sum-dpl-302-m/devset_images/devset_images"

# Hàm để đọc và xử lý từng ảnh
def load_and_preprocess_image(image_path, target_size=(256, 256)):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image = np.array(image)
    if image.shape[-1] == 4:  # Nếu ảnh có kênh alpha (RGBA), chuyển về RGB
        image = image[:, :, :3]
    image = image / 255.0  # Chuẩn hóa ảnh về khoảng [0, 1]
    return image

# Đọc tất cả các ảnh trong folder
images = []
Y_train = []

# List of allowed file extensions
allowed_extensions = ['.jpg', '.png', '.gif']

# Giả sử label_array là một mảng các tên file và nhãn tương ứng
# Cập nhật lại danh sách các file ảnh với các định dạng khác nhau
for filename in os.listdir(folder_path):
    # Check if the file has one of the allowed extensions
    if os.path.splitext(filename)[1].lower() in allowed_extensions:
        # Extract the base name without the extension to match with label_array
        basename = os.path.splitext(filename)[0]
        # Find the corresponding label for the current image file
        for i in label_array:
            if basename == str(i[0]):
                image_path = os.path.join(folder_path, filename)
                image = load_and_preprocess_image(image_path)
                images.append(image)
                Y_train.append(i[1])
                break  # Break the inner loop once the label is found

In [7]:
images = np.array(images)
Y_train = np.array(Y_train)

In [8]:
train_images, val_images, train_labels, val_labels = train_test_split(images, Y_train, test_size=0.2, random_state=42)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.applications import ResNet152V2

In [10]:
model = Sequential()

In [11]:
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

234545216/234545216 [==============================] - 14s 0us/step


In [12]:
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
base_model.trainable = False

In [13]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 8, 8, 2048)        58331648  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 512)               67109376  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 125,441,537
Trainable params: 67,109,889
Non-trainable params: 58,331,648
_________________________________________________________________


In [15]:
history = model.fit(
    train_images, train_labels,
    epochs= 2,
    batch_size=32,
    validation_data=(val_images, val_labels)
)

Epoch 1/2
132/132 [==============================] - 813s 6s/step - loss: 2.7227 - accuracy: 0.8336 - val_loss: 1.1627 - val_accuracy: 0.8769
Epoch 2/2
132/132 [==============================] - 865s 7s/step - loss: 0.9632 - accuracy: 0.9029 - val_loss: 0.9484 - val_accuracy: 0.8873


In [16]:
loss, accuracy = model.evaluate(val_images, val_labels)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

33/33 [==============================] - 146s 4s/step - loss: 0.9484 - accuracy: 0.8873
Validation loss: 0.9483863115310669
Validation accuracy: 0.8873106241226196


In [17]:
# Đường dẫn tới thư mục chứa ảnh
folder_path = "C:/Users/minhd/FPTU lab/DPL302m/Kaggle/2024-sum-dpl-302-m/testset_images/testset_images"

# Đọc và xử lý ảnh, sau đó dự đoán với mô hình
def predict_image(model, img_path, target_size=(256, 256)):
    image = Image.open(img_path)
    image = image.resize(target_size)
    image = np.array(image)
    if image.shape[-1] == 4:  # Nếu ảnh có kênh alpha (RGBA), chuyển về RGB
        image = image[:, :, :3]
    image = image / 255.0
    image = np.expand_dims(image, axis=0)  # Thêm chiều batch vào hình ảnh
    prediction = model.predict(image)
    return 1 if prediction[0][0] >= 0.5 else 0

test_label = []
id_test = []

# Danh sách các phần mở rộng file hợp lệ
allowed_extensions = ['.jpg', '.png', '.gif']

# Giả sử label_array là một mảng các tên file và nhãn tương ứng
# Cập nhật lại danh sách các file ảnh với các định dạng khác nhau
for filename in os.listdir(folder_path):
    # Kiểm tra nếu file có phần mở rộng hợp lệ
    if os.path.splitext(filename)[1].lower() in allowed_extensions:
        # Lấy tên file không có phần mở rộng để khớp với label_array
        basename = os.path.splitext(filename)[0]
        id_test.append(basename)
        # Tìm nhãn tương ứng cho file ảnh hiện tại
        image_path = os.path.join(folder_path, filename)
        output = predict_image(model, image_path)
        test_label.append(output)

print(test_label)

1/1 [==============================] - 0s 269ms/step
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1

In [18]:
submit = pd.DataFrame({'id': id_test, 'label': test_label})
submit.head()

,id,label
0,100167480,1
1,10021457945,0
2,10025866403,0
3,10035741293,0
4,10126331405,0


In [19]:
results_csv_path = 'submitResnet152V2.csv'
submit.to_csv(results_csv_path, index=False)